In [1]:
from fastsam import FastSAM, FastSAMPrompt
import torch
import lovely_tensors
from PIL import Image
import numpy as np
import math
import time
import cv2
import onnxruntime
lovely_tensors.monkey_patch()

model = FastSAM('./weights/FastSAM-s.pt')
IMAGE_PATH = './images/parrot.jpg'
DEVICE = 'mps'

print(model.model)
for _ in range(10):
    everything_results = model(IMAGE_PATH, device=DEVICE, retina_masks=True, imgsz=1024, conf=0.4, iou=0.9,)

prompt_process = FastSAMPrompt(IMAGE_PATH, everything_results, device=DEVICE)

# everything prompt
ann = prompt_process.everything_prompt()

prompt_process.plot(annotations=ann,output_path='./output/cat.jpg',)

SegmentationModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
 

/Users/mateuszsluszniak/Desktop/AI/sam_research/FastSAM/ultralytics/yolo/utils/torch_utils.py:94: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  elif mps and getattr(torch, 'has_mps', False) and torch.backends.mps.is_available() and TORCH_2_0:

image 1/1 /Users/mateuszsluszniak/Desktop/AI/sam_research/FastSAM/images/parrot.jpg: 1024x768 69 objects, 215.9ms
Speed: 8.8ms preprocess, 215.9ms inference, 152.9ms postprocess per image at shape (1, 3, 1024, 1024)
/Users/mateuszsluszniak/Desktop/AI/sam_research/FastSAM/ultralytics/yolo/utils/torch_utils.py:94: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  elif mps and getattr(torch, 'has_mps', False) and torch.backends.mps.is_available() and TORCH_2_0:

image 1/1 /Users/mateuszsluszniak/Desktop/AI/sam_research/FastSAM/images/parrot.jpg: 1024x768 69 objects, 33.9ms
Speed: 2.6ms preprocess, 33.9ms inference, 7.8ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1

In [4]:
type(model.model)

ultralytics.nn.tasks.SegmentationModel

In [5]:
from fastsam import FastSAMDecoder
decoder = FastSAMDecoder(model)
x = np.array(Image.open(IMAGE_PATH))
res = decoder.run_encoder(x)

TypeError: 'NoneType' object is not subscriptable

In [6]:
res

NameError: name 'res' is not defined

In [7]:
x = np.array(Image.open(IMAGE_PATH))
res = decoder.run_encoder(x)
print(res)
# print(res.shape)

print(res.masks.masks.shape)
# print(x.shape)
# x = torch.Tensor(x)
# x = torch.permute(x, (2, 0, 1))
# x = x.unsqueeze(0)
# print(x.size())
# x = x[:, :, :1920, :1024]
# print(model.model(x)[0].numpy())

TypeError: 'NoneType' object is not subscriptable

In [8]:
from ultralytics.yolo.utils.ops import non_max_suppression


CLASSES = list(range(1000))
im = cv2.imread(IMAGE_PATH)
img0 = im.copy()
im = cv2.resize(im, (1024, 1024), interpolation = cv2.INTER_AREA)
# Convert
im = im.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
im = np.ascontiguousarray(im)
# Convert into torch
im = torch.from_numpy(im)
im = im.float()  # uint8 to fp16/32
im /= 255  # 0 - 255 to 0.0 - 1.0
if len(im.shape) == 3:
    im = im[None]  # expand for batch dim

# Load the compiled model
# model = torch.jit.load('model_converted.pt')

# Inference
pred = model.model(im)
print(pred)
pred = non_max_suppression(pred) #nms function used same as yolov5 detect.py

#Process predictions
for i, det in enumerate(pred):  # per image
    im0 = img0.copy()
    color=(30, 30, 30)
    txt_color=(255, 255, 255)
    h_size, w_size = im.shape[-2:]
    # print(h_size, w_size)
    lw = max(round(sum(im.shape) / 2 * 0.003), 2) 

    if len(det):
        # Write results
        for *xyxy, conf, cls in reversed(det):
            c = int(cls)  # integer class
            label = f'{CLASSES[c]} {conf:.2f}'
            print(label)
            box = xyxy
            print(box)
            p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
            cv2.rectangle(im0, p1, p2, color, thickness=lw, lineType=cv2.LINE_AA)
            tf = max(lw - 1, 1)  # font thickness
            w, h = cv2.getTextSize(label, 0, fontScale=lw / 3, thickness=tf)[0]  # text width, height
            outside = p1[1] - h - 3 >= 0  # label fits outside box
            p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
            print(p1, p2)
            cv2.rectangle(im0, p1, p2, color, -1, cv2.LINE_AA)  # filled
            cv2.putText(im0,
                        label, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (255, 0, 0),
                        thickness=2,
                        lineType=cv2.LINE_AA)
    # Save results (image with detections)
    # cv2.putText(im0,
    #                     label, (50, 50),
    #                     cv2.FONT_HERSHEY_SIMPLEX,
    #                     1,
    #                     (255, 0, 0),
    #                     thickness=2,
    #                     lineType=cv2.LINE_AA)
    status = cv2.imwrite('out.jpg', im0)
    print("status ", status)

RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [10]:
from ultralytics.yolo.v8.segment.predict import *
from ultralytics.yolo.utils import ops
import torch
from ultralytics.nn.tasks import attempt_load_one_weight
import cv2
import numpy as np
# state_dict = torch.load('./weights/FastSAM-s.pt')
# model = FastSAM.load_state_dict(state_dict)

model, _ = attempt_load_one_weight('./weights/FastSAM-s.pt')
IMAGE_PATH = './images/parrot.jpg'
im = cv2.imread(IMAGE_PATH)
img0 = im.copy()
im = cv2.resize(im, (1024, 1024), interpolation = cv2.INTER_AREA)
im = im.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
im = np.ascontiguousarray(im)
# Convert into torch
im = torch.from_numpy(im)
im = im.float()  # uint8 to fp16/32
im /= 255  # 0 - 255 to 0.0 - 1.0
if len(im.shape) == 3:
    im = im[None]  # expand for batch dim

print(im.size())

# sp = SegmentationPredictor(overrides=model.overrides, _callbacks=model.callbacks)
sp = SegmentationPredictor()
# sp.setup_model(model.model)
sp.device = 'mps'
model.eval()
sp.batch = "image_test.jpg"
x = model(im)
x
im.permute((0, 3, 1, 2))
def postprocess(sp, preds, img, orig_imgs):
    """TODO: filter by classes."""
    p = ops.non_max_suppression(preds[0],
                                sp.args.conf,
                                sp.args.iou,
                                agnostic=sp.args.agnostic_nms,
                                max_det=sp.args.max_det,
                                nc=len(model.names),
                                classes=sp.args.classes)
    results = []
    proto = preds[1][-1] if len(preds[1]) == 3 else preds[1]  # second output is len 3 if pt, but only 1 if exported
    for i, pred in enumerate(p):
        orig_img = orig_imgs[i] if isinstance(orig_imgs, list) else orig_imgs
        path = sp.batch[0]
        img_path = path[i] if isinstance(path, list) else path
        if not len(pred):  # save empty boxes
            results.append(Results(orig_img=orig_img, path=img_path, names=model.names, boxes=pred[:, :6]))
            continue
        if sp.args.retina_masks:
            if not isinstance(orig_imgs, torch.Tensor):
                pred[:, :4] = ops.scale_boxes(img.shape[2:], pred[:, :4], orig_img.shape)
            masks = ops.process_mask_native(proto[i], pred[:, 6:], pred[:, :4], orig_img.shape[:2])  # HWC
        else:
            masks = ops.process_mask(proto[i], pred[:, 6:], pred[:, :4], img.shape[2:], upsample=True)  # HWC
            if not isinstance(orig_imgs, torch.Tensor):
                pred[:, :4] = ops.scale_boxes(img.shape[2:], pred[:, :4], orig_img.shape)
        results.append(
            Results(orig_img=orig_img, path=img_path, names=model.names, boxes=pred[:, :6], masks=masks))
    return results

print("sp.args.cond", sp.args.conf)
print("sp.args.iou", sp.args.iou)
print("sp.args.agnostic_nms", sp.args.agnostic_nms)
print("sp.args.max_det", sp.args.max_det)
print("model.model.names", model.names)
print("sp.args.classes", sp.args.classes)
print("sp.batch[0]", sp.batch[0])
print("sp.args.retina_masks", sp.args.retina_masks)

res = postprocess(sp, x, im, im)
# res = sp.postprocess(x, im, im)
print(res)

torch.Size([1, 3, 1024, 1024])


RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [29]:
import os
os.getcwd()

'/Users/mateuszsluszniak/Desktop/AI/sam_research/FastSAM'

In [27]:
# print(sp.args.retina_masks)

False


: 

In [27]:
# initialized only for plotting 
prompt_process = FastSAMPrompt(IMAGE_PATH, res, device=DEVICE)

# everything prompt
ann = res[0].masks.data
# ann = prompt_process.everything_prompt()


prompt_process.plot(annotations=ann,output_path='./output/parrot.jpg',)

In [6]:
x = torch.rand((1, 3, 1024, 1024))
model.model.eval()
res1, res2 = model.model(x)

In [7]:
res1

tensor[1, 37, 21504] n=795648 (3.0Mb) x∈[-4.462, 1.022e+03] μ=29.680 σ=136.337

In [8]:
res2

([tensor[1, 65, 128, 128] n=1064960 (4.1Mb) x∈[-11.530, 13.182] μ=0.858 σ=3.624,
  tensor[1, 65, 64, 64] n=266240 (1.0Mb) x∈[-12.244, 11.366] μ=0.854 σ=3.004,
  tensor[1, 65, 32, 32] n=66560 (0.3Mb) x∈[-14.146, 8.306] μ=0.854 σ=2.298],
 tensor[1, 32, 21504] n=688128 (2.6Mb) x∈[-4.462, 2.603] μ=-0.115 σ=0.843,
 tensor[1, 32, 256, 256] n=2097152 (8Mb) x∈[-0.278, 5.295] μ=0.835 σ=0.729)

In [9]:
model.point_prompt([[200, 400]], [1])

AttributeError: 'FastSAM' object has no attribute 'point_prompt'. See valid attributes below.
None

In [5]:
model.names

{0: 'object'}

In [10]:
import math
import time
import cv2
import numpy as np
import onnxruntime

import numpy as np
import cv2

class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
               'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
               'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
               'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
               'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
               'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
               'scissors', 'teddy bear', 'hair drier', 'toothbrush']

# Create a list of colors for each class where each color is a tuple of 3 integer values
rng = np.random.default_rng(3)
colors = rng.uniform(0, 255, size=(len(class_names), 3))


def nms(boxes, scores, iou_threshold):
    # Sort by score
    sorted_indices = np.argsort(scores)[::-1]

    keep_boxes = []
    while sorted_indices.size > 0:
        # Pick the last box
        box_id = sorted_indices[0]
        keep_boxes.append(box_id)

        # Compute IoU of the picked box with the rest
        ious = compute_iou(boxes[box_id, :], boxes[sorted_indices[1:], :])

        # Remove boxes with IoU over the threshold
        keep_indices = np.where(ious < iou_threshold)[0]

        # print(keep_indices.shape, sorted_indices.shape)
        sorted_indices = sorted_indices[keep_indices + 1]

    return keep_boxes


def compute_iou(box, boxes):
    # Compute xmin, ymin, xmax, ymax for both boxes
    xmin = np.maximum(box[0], boxes[:, 0])
    ymin = np.maximum(box[1], boxes[:, 1])
    xmax = np.minimum(box[2], boxes[:, 2])
    ymax = np.minimum(box[3], boxes[:, 3])

    # Compute intersection area
    intersection_area = np.maximum(0, xmax - xmin) * np.maximum(0, ymax - ymin)

    # Compute union area
    box_area = (box[2] - box[0]) * (box[3] - box[1])
    boxes_area = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
    union_area = box_area + boxes_area - intersection_area

    # Compute IoU
    iou = intersection_area / union_area

    return iou


def xywh2xyxy(x):
    # Convert bounding box (x, y, w, h) to bounding box (x1, y1, x2, y2)
    y = np.copy(x)
    y[..., 0] = x[..., 0] - x[..., 2] / 2
    y[..., 1] = x[..., 1] - x[..., 3] / 2
    y[..., 2] = x[..., 0] + x[..., 2] / 2
    y[..., 3] = x[..., 1] + x[..., 3] / 2
    return y


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def draw_detections(image, boxes, scores, class_ids, mask_alpha=0.3, mask_maps=None):
    img_height, img_width = image.shape[:2]
    size = min([img_height, img_width]) * 0.0006
    text_thickness = int(min([img_height, img_width]) * 0.001)

    mask_img = draw_masks(image, boxes, class_ids, mask_alpha, mask_maps)

    # Draw bounding boxes and labels of detections
    for box, score, class_id in zip(boxes, scores, class_ids):
        color = colors[class_id]

        x1, y1, x2, y2 = box.astype(int)

        # Draw rectangle
        cv2.rectangle(mask_img, (x1, y1), (x2, y2), color, 2)

        label = class_names[class_id]
        caption = f'{label} {int(score * 100)}%'
        (tw, th), _ = cv2.getTextSize(text=caption, fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                                      fontScale=size, thickness=text_thickness)
        th = int(th * 1.2)

        cv2.rectangle(mask_img, (x1, y1),
                      (x1 + tw, y1 - th), color, -1)

        cv2.putText(mask_img, caption, (x1, y1),
                    cv2.FONT_HERSHEY_SIMPLEX, size, (255, 255, 255), text_thickness, cv2.LINE_AA)

    return mask_img


def draw_masks(image, boxes, class_ids, mask_alpha=0.3, mask_maps=None):
    mask_img = image.copy()

    # Draw bounding boxes and labels of detections
    for i, (box, class_id) in enumerate(zip(boxes, class_ids)):
        color = colors[class_id]

        x1, y1, x2, y2 = box.astype(int)

        # Draw fill mask image
        if mask_maps is None:
            cv2.rectangle(mask_img, (x1, y1), (x2, y2), color, -1)
        else:
            crop_mask = mask_maps[i][y1:y2, x1:x2, np.newaxis]
            crop_mask_img = mask_img[y1:y2, x1:x2]
            crop_mask_img = crop_mask_img * (1 - crop_mask) + crop_mask * color
            mask_img[y1:y2, x1:x2] = crop_mask_img

    return cv2.addWeighted(mask_img, mask_alpha, image, 1 - mask_alpha, 0)


def draw_comparison(img1, img2, name1, name2, fontsize=2.6, text_thickness=3):
    (tw, th), _ = cv2.getTextSize(text=name1, fontFace=cv2.FONT_HERSHEY_DUPLEX,
                                  fontScale=fontsize, thickness=text_thickness)
    x1 = img1.shape[1] // 3
    y1 = th
    offset = th // 5
    cv2.rectangle(img1, (x1 - offset * 2, y1 + offset),
                  (x1 + tw + offset * 2, y1 - th - offset), (0, 115, 255), -1)
    cv2.putText(img1, name1,
                (x1, y1),
                cv2.FONT_HERSHEY_DUPLEX, fontsize,
                (255, 255, 255), text_thickness)

    (tw, th), _ = cv2.getTextSize(text=name2, fontFace=cv2.FONT_HERSHEY_DUPLEX,
                                  fontScale=fontsize, thickness=text_thickness)
    x1 = img2.shape[1] // 3
    y1 = th
    offset = th // 5
    cv2.rectangle(img2, (x1 - offset * 2, y1 + offset),
                  (x1 + tw + offset * 2, y1 - th - offset), (94, 23, 235), -1)

    cv2.putText(img2, name2,
                (x1, y1),
                cv2.FONT_HERSHEY_DUPLEX, fontsize,
                (255, 255, 255), text_thickness)

    combined_img = cv2.hconcat([img1, img2])
    if combined_img.shape[1] > 3840:
        combined_img = cv2.resize(combined_img, (3840, 2160))

    return combined_img


class YOLOSeg:

    def __init__(self, path, conf_thres=0.7, iou_thres=0.5, num_masks=32):
        self.conf_threshold = conf_thres
        self.iou_threshold = iou_thres
        self.num_masks = num_masks

        # Initialize model
        self.initialize_model(path)

    def __call__(self, image):
        return self.segment_objects(image)

    def initialize_model(self, path):
        self.session = onnxruntime.InferenceSession(path,
                                                    providers=['CUDAExecutionProvider',
                                                               'CPUExecutionProvider'])
        # Get model info
        self.get_input_details()
        self.get_output_details()

    def segment_objects(self, image):
        input_tensor = self.prepare_input(image)

        # Perform inference on the image
        outputs = self.inference(input_tensor)

        self.boxes, self.scores, self.class_ids, mask_pred = self.process_box_output(outputs[0])
        self.mask_maps = self.process_mask_output(mask_pred, outputs[1])

        return self.boxes, self.scores, self.class_ids, self.mask_maps

    def prepare_input(self, image):
        self.img_height, self.img_width = image.shape[:2]

        input_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Resize input image
        input_img = cv2.resize(input_img, (self.input_width, self.input_height))

        # Scale input pixel values to 0 to 1
        input_img = input_img / 255.0
        input_img = input_img.transpose(2, 0, 1)
        input_tensor = input_img[np.newaxis, :, :, :].astype(np.float32)

        return input_tensor

    def inference(self, input_tensor):
        start = time.perf_counter()
        outputs = self.session.run(self.output_names, {self.input_names[0]: input_tensor})

        # print(f"Inference time: {(time.perf_counter() - start)*1000:.2f} ms")
        return outputs

    def process_box_output(self, box_output):

        predictions = np.squeeze(box_output).T
        num_classes = box_output.shape[1] - self.num_masks - 4

        # Filter out object confidence scores below threshold
        scores = np.max(predictions[:, 4:4+num_classes], axis=1)
        predictions = predictions[scores > self.conf_threshold, :]
        scores = scores[scores > self.conf_threshold]

        if len(scores) == 0:
            return [], [], [], np.array([])

        box_predictions = predictions[..., :num_classes+4]
        mask_predictions = predictions[..., num_classes+4:]

        # Get the class with the highest confidence
        class_ids = np.argmax(box_predictions[:, 4:], axis=1)

        # Get bounding boxes for each object
        boxes = self.extract_boxes(box_predictions)

        # Apply non-maxima suppression to suppress weak, overlapping bounding boxes
        indices = nms(boxes, scores, self.iou_threshold)

        return boxes[indices], scores[indices], class_ids[indices], mask_predictions[indices]

    def process_mask_output(self, mask_predictions, mask_output):

        if mask_predictions.shape[0] == 0:
            return []

        mask_output = np.squeeze(mask_output)

        # Calculate the mask maps for each box
        num_mask, mask_height, mask_width = mask_output.shape  # CHW
        masks = sigmoid(mask_predictions @ mask_output.reshape((num_mask, -1)))
        masks = masks.reshape((-1, mask_height, mask_width))

        # Downscale the boxes to match the mask size
        scale_boxes = self.rescale_boxes(self.boxes,
                                   (self.img_height, self.img_width),
                                   (mask_height, mask_width))

        # For every box/mask pair, get the mask map
        mask_maps = np.zeros((len(scale_boxes), self.img_height, self.img_width))
        blur_size = (int(self.img_width / mask_width), int(self.img_height / mask_height))
        for i in range(len(scale_boxes)):

            scale_x1 = int(math.floor(scale_boxes[i][0]))
            scale_y1 = int(math.floor(scale_boxes[i][1]))
            scale_x2 = int(math.ceil(scale_boxes[i][2]))
            scale_y2 = int(math.ceil(scale_boxes[i][3]))

            x1 = int(math.floor(self.boxes[i][0]))
            y1 = int(math.floor(self.boxes[i][1]))
            x2 = int(math.ceil(self.boxes[i][2]))
            y2 = int(math.ceil(self.boxes[i][3]))

            scale_crop_mask = masks[i][scale_y1:scale_y2, scale_x1:scale_x2]
            crop_mask = cv2.resize(scale_crop_mask,
                              (x2 - x1, y2 - y1),
                              interpolation=cv2.INTER_CUBIC)

            crop_mask = cv2.blur(crop_mask, blur_size)

            crop_mask = (crop_mask > 0.5).astype(np.uint8)
            mask_maps[i, y1:y2, x1:x2] = crop_mask

        return mask_maps

    def extract_boxes(self, box_predictions):
        # Extract boxes from predictions
        boxes = box_predictions[:, :4]

        # Scale boxes to original image dimensions
        boxes = self.rescale_boxes(boxes,
                                   (self.input_height, self.input_width),
                                   (self.img_height, self.img_width))

        # Convert boxes to xyxy format
        boxes = xywh2xyxy(boxes)

        # Check the boxes are within the image
        boxes[:, 0] = np.clip(boxes[:, 0], 0, self.img_width)
        boxes[:, 1] = np.clip(boxes[:, 1], 0, self.img_height)
        boxes[:, 2] = np.clip(boxes[:, 2], 0, self.img_width)
        boxes[:, 3] = np.clip(boxes[:, 3], 0, self.img_height)

        return boxes

    def draw_detections(self, image, draw_scores=True, mask_alpha=0.4):
        return draw_detections(image, self.boxes, self.scores,
                               self.class_ids, mask_alpha)

    def draw_masks(self, image, draw_scores=True, mask_alpha=0.5):
        return draw_detections(image, self.boxes, self.scores,
                               self.class_ids, mask_alpha, mask_maps=self.mask_maps)

    def get_input_details(self):
        model_inputs = self.session.get_inputs()
        self.input_names = [model_inputs[i].name for i in range(len(model_inputs))]

        self.input_shape = model_inputs[0].shape
        self.input_height = self.input_shape[2]
        self.input_width = self.input_shape[3]

    def get_output_details(self):
        model_outputs = self.session.get_outputs()
        self.output_names = [model_outputs[i].name for i in range(len(model_outputs))]

    @staticmethod
    def rescale_boxes(boxes, input_shape, image_shape):
        # Rescale boxes to original image dimensions
        input_shape = np.array([input_shape[1], input_shape[0], input_shape[1], input_shape[0]])
        boxes = np.divide(boxes, input_shape, dtype=np.float32)
        boxes *= np.array([image_shape[1], image_shape[0], image_shape[1], image_shape[0]])

        return boxes

In [14]:
help(model)

Help on FastSAM in module fastsam.model object:

class FastSAM(ultralytics.yolo.engine.model.YOLO)
 |  FastSAM(model: Union[str, pathlib.Path] = 'yolov8n.pt', task=None) -> None
 |  
 |  Method resolution order:
 |      FastSAM
 |      ultralytics.yolo.engine.model.YOLO
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__(self, source=None, stream=False, **kwargs)
 |      Calls the 'predict' function with given arguments to perform object detection.
 |  
 |  __getattr__(self, attr)
 |      Raises error if object has no requested attribute.
 |  
 |  export(self, **kwargs)
 |      Export model.
 |      
 |      Args:
 |          **kwargs : Any other args accepted by the predictors. To see all args check 'configuration' section in docs
 |  
 |  info(self, detailed=False, verbose=True)
 |      Logs model info.
 |      
 |      Args:
 |          detailed (bool): Show detailed information about model.
 |          verbose (bool): Controls verbosity.
 |  
 |  predict(self, 

In [13]:
def process_mask_output(self, mask_predictions, mask_output):

    if mask_predictions.shape[0] == 0:
        return []

    mask_output = np.squeeze(mask_output)

    # Calculate the mask maps for each box
    num_mask, mask_height, mask_width = mask_output.shape  # CHW
    masks = sigmoid(mask_predictions @ mask_output.reshape((num_mask, -1)))
    masks = masks.reshape((-1, mask_height, mask_width))

    # Downscale the boxes to match the mask size
    scale_boxes = self.rescale_boxes(self.boxes,
                                (self.img_height, self.img_width),
                                (mask_height, mask_width))

    # For every box/mask pair, get the mask map
    mask_maps = np.zeros((len(scale_boxes), self.img_height, self.img_width))
    blur_size = (int(self.img_width / mask_width), int(self.img_height / mask_height))
    for i in range(len(scale_boxes)):

        scale_x1 = int(math.floor(scale_boxes[i][0]))
        scale_y1 = int(math.floor(scale_boxes[i][1]))
        scale_x2 = int(math.ceil(scale_boxes[i][2]))
        scale_y2 = int(math.ceil(scale_boxes[i][3]))

        x1 = int(math.floor(self.boxes[i][0]))
        y1 = int(math.floor(self.boxes[i][1]))
        x2 = int(math.ceil(self.boxes[i][2]))
        y2 = int(math.ceil(self.boxes[i][3]))

        scale_crop_mask = masks[i][scale_y1:scale_y2, scale_x1:scale_x2]
        crop_mask = cv2.resize(scale_crop_mask,
                            (x2 - x1, y2 - y1),
                            interpolation=cv2.INTER_CUBIC)

        crop_mask = cv2.blur(crop_mask, blur_size)

        crop_mask = (crop_mask > 0.5).astype(np.uint8)
        mask_maps[i, y1:y2, x1:x2] = crop_mask

    return mask_maps

AttributeError: 'FastSAM' object has no attribute 'img_height'. See valid attributes below.
None

In [22]:
print(model.__dict__)

{'callbacks': defaultdict(<class 'list'>, {'on_pretrain_routine_start': [<function on_pretrain_routine_start at 0x3139239d0>], 'on_pretrain_routine_end': [<function on_pretrain_routine_end at 0x313923af0>], 'on_train_start': [<function on_train_start at 0x313923b80>], 'on_train_epoch_start': [<function on_train_epoch_start at 0x313923c10>], 'on_train_batch_start': [<function on_train_batch_start at 0x313923ca0>], 'optimizer_step': [<function optimizer_step at 0x313923d30>], 'on_before_zero_grad': [<function on_before_zero_grad at 0x313923dc0>], 'on_train_batch_end': [<function on_train_batch_end at 0x313923e50>], 'on_train_epoch_end': [<function on_train_epoch_end at 0x313923ee0>], 'on_fit_epoch_end': [<function on_fit_epoch_end at 0x313923f70>], 'on_model_save': [<function on_model_save at 0x31392e040>], 'on_train_end': [<function on_train_end at 0x31392e0d0>], 'on_params_update': [<function on_params_update at 0x31392e160>], 'teardown': [<function teardown at 0x31392e1f0>], 'on_val_s